In [472]:
import pandas as pd
import numpy as np
import tabula
import os

In [481]:
FOLDER = os.path.join(r'C:\\',"Users","ChunSangWoo","Documents","New Jersey","Monthly Data")
FILE_NAME_CONFIG = {
    'Sports Wagering Handle':{
        'pages':'6',
        'table_rows':3,
        'table_index':1,
        'cols_to_drop':[2],
        'do_cleaning': True,  #True for SB Handle; False for SB Revenue
        'null_col': False,
        'concat': False,

    },
    'Sports Wagering Revenue':{
        'pages':'2',
        'table_rows':8,
        'table_index':0,
        'cols_to_drop':[1,3,4,5,6],
        'do_cleaning' : False,
        'null_col': True,  #True for SB Revenue; False for SB Handle
        'concat': False,

    },
    'Monthly SB Figures':{
        'pages':[],
        'table_rows':[],
        'table_index':[],
        'cols_to_drop':[],
        'do_cleaning' : False,
        'null_col': False,  #True for SB Revenue; False for SB Handle
        'concat': True,
    }
}

def load_raw_dfs(filename, pages, folder=FOLDER):
    print(locals())
    path = os.path.join(folder, filename)
    #path = r'C:\Users\ChunSangWoo\Documents\New Jersey/November2019.pdf'
    
    #dfs = tabula.read_pdf(path, pages=pages, multiple_tables=True, stream=True)
    dfs = tabula.read_pdf(path, pages=pages, multiple_tables=True, stream=True, guess=True)

    return dfs

In [482]:
def load_clean_df(dataset_name, filename, folder=FOLDER):
    raw_dfs = load_raw_dfs(filename, pages=FILE_NAME_CONFIG[dataset_name]['pages']) #list of multiple dataframes
    raw_df = raw_dfs[FILE_NAME_CONFIG[dataset_name]['table_index']]
    df=raw_df
    
    if FILE_NAME_CONFIG[dataset_name]['do_cleaning']:
        #df.drop(df.columns[FILE_NAME_CONFIG[dataset_name]['cols_to_drop']], axis=1, inplace=True)
        
        df.columns = ['','Monthly Handle','']
        df=df[1:FILE_NAME_CONFIG[dataset_name]['table_rows']+1]
        df.set_index(df.columns[0], inplace=True)
        df.columns = [filename]
        df = df.transpose()
        df.columns = ['Retail Handle','Digital Handle','Total Handle']

        for col_name in df.columns:
            df[col_name]=df[col_name].str.replace(',','').astype(float)
            
    if FILE_NAME_CONFIG[dataset_name]['null_col']:
        
        df.drop(df.columns[[1,3,4,5,6]], axis=1, inplace=True)  #HOW DO I USE CONFIG: 'cols_to_drop' INSTEAD OF TYPING OUT THE COLUMNS TO DROP?
        df.columns = df.iloc[0]
        df = df[8:9]
        df.columns = ['','Total Revenue']

        df.set_index(df.columns[0], inplace=True)
        df.columns = [filename]
        df = df.transpose()

        for col_name in df.columns:
            df[col_name]=df[col_name].str.replace(',','').astype(float)

    return(df)

def monthly_df(dataset_name, filename, folder=FOLDER):
    monthly_df = pd.concat([df1, df2], axis=1)
    
    return(monthly_df)

In [483]:
load_raw_dfs('1911.pdf', pages='6')[1]

{'folder': 'C:\\\\Users\\ChunSangWoo\\Documents\\New Jersey\\Monthly Data', 'pages': '6', 'filename': '1911.pdf'}


,0,1,2
0,Sports Wagering Handle,Monthly Handle,YTD Handle
1,On-site (Retail) Sports Wagering,"75,890,898","678,523,405"
2,Internet (On-Line) Sports Wagering,"486,784,645","3,346,632,365"
3,Total Sports Wagering Handle,"562,675,543","4,025,155,770"


In [484]:
load_clean_df('Sports Wagering Handle', '1911.pdf')

{'folder': 'C:\\\\Users\\ChunSangWoo\\Documents\\New Jersey\\Monthly Data', 'pages': '6', 'filename': '1911.pdf'}


,Retail Handle,Digital Handle,Total Handle
1911.pdf,75890898.0,486784645.0,562675543.0


In [485]:
load_clean_df('Sports Wagering Revenue', '1911.pdf')

{'folder': 'C:\\\\Users\\ChunSangWoo\\Documents\\New Jersey\\Monthly Data', 'pages': '2', 'filename': '1911.pdf'}


,Sports Wagering Revenue
1911.pdf,32895546.0


In [489]:
monthly_df('Monthly SB Figures', '1911.pdf')

,Retail Handle,Digital Handle,Total Handle,Sports Wagering Revenue
November2019.pdf,75890898.0,486784645.0,562675543.0,32895546.0


In [490]:
import os

for filename in FOLDER:
    if filename.endswith(".PDF"): 
        continue
    else:
        continue